<a href="https://colab.research.google.com/github/svarunid/Dog-Breed-Identification-using-TfHub-Model/blob/main/recipeye_ingredient_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛒 Grocery Items Classification
Preparing a DL model to classify itemsin a grocery store.

## Data
The dataset from https://github.com/marcusklasson/GroceryStoreDataset contains images of fruits, vegetables and other packaged itemscommonly found in the grocery stores with their labels. The data is separated into train and test sets.


## Evaluation
For each image in the test set, predict a probability for each of the different classes.

The evaluation will be made based on Multi Class Log Loss between 'Predicted probs' and 'Observed labels'.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Data Loading
1. Read 'classes.csv' file and get the labels and label id mappings.
2. Read the 'train.csv' and 'test.csv' to get paths of train & test images with associated labels.

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Recipeye/GroceryStoreDataset/dataset/classes.csv")
df = df[["Coarse Class Name (str)","Coarse Class ID (int)"]]
df = df.drop_duplicates()
df = df.set_index("Coarse Class ID (int)")
labels_dict = df.to_dict()["Coarse Class Name (str)"]

In [3]:
labels_dict

{0: 'Apple',
 1: 'Avocado',
 2: 'Banana',
 3: 'Kiwi',
 4: 'Lemon',
 5: 'Lime',
 6: 'Mango',
 7: 'Melon',
 8: 'Nectarine',
 9: 'Orange',
 10: 'Papaya',
 11: 'Passion-Fruit',
 12: 'Peach',
 13: 'Pear',
 14: 'Pineapple',
 15: 'Plum',
 16: 'Pomegranate',
 17: 'Red-Grapefruit',
 18: 'Satsumas',
 19: 'Juice',
 20: 'Milk',
 21: 'Oatghurt',
 22: 'Oat-Milk',
 23: 'Sour-Cream',
 24: 'Sour-Milk',
 25: 'Soyghurt',
 26: 'Soy-Milk',
 27: 'Yoghurt',
 28: 'Asparagus',
 29: 'Aubergine',
 30: 'Cabbage',
 31: 'Carrots',
 32: 'Cucumber',
 33: 'Garlic',
 34: 'Ginger',
 35: 'Leek',
 36: 'Mushroom',
 37: 'Onion',
 38: 'Pepper',
 39: 'Potato',
 40: 'Red-Beet',
 41: 'Tomato',
 42: 'Zucchini'}

In [4]:
def import_df(path):
  # txt as csv -> select specific cols -> name them.
  df = pd.read_csv(path,
                   header = None,
                   names = ["img_path", "coarse_id"],
                   usecols=[0,2])
  # Shuffle the dataset
  df = df.sample(frac=1, random_state=1).reset_index(drop=True)
  return df

In [5]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Recipeye/GroceryStoreDataset/dataset/"

In [6]:
train_df = import_df(data_path + "train.txt")
test_df = import_df(data_path + "test.txt")

In [7]:
labels = np.array(list(labels_dict.keys()))

# Split Data
The data is split into train and validation set for training and validation.

In [8]:
X = [(data_path + img_path) for img_path in list(train_df["img_path"])]
y = [labels == label for label in train_df["coarse_id"].to_numpy()]

In [60]:
from sklearn.model_selection import train_test_split 

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)
X_test = [(data_path + img_path) for img_path in list(test_df["img_path"])]
y_test = [labels == label for label in test_df["coarse_id"].to_numpy()]

# Preprocessing
Define a funtion to preprocess images
1. Read images from gdrive as 3 channel img tensors.
2. Noramlize tensors.
3. Resize the tensors to a specific size based on the model used.


In [10]:
# The size to which the image has to be resized
IMG_SIZE = 224

def img_to_tensor(img_path,img_size=IMG_SIZE):
  """
  Fetches the image from the file path, turns it into tensors
  and scale them to the size specified.
  """
  # Fetch the image from the img_path path
  image = tf.io.read_file(img_path)
  # Covert the image to tensors of 3 channel
  image = tf.image.decode_jpeg(image, channels=3)
  # Normalize the tensor to have values ranging from 0(0) to 1(255)
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to the given size
  image = tf.image.resize(image, size=[img_size,img_size])

  return image

# Batching

In [65]:
# The size to each batch
BATCH_SIZE = 32

def get_img_label(img_path, label):
  """
  Takes an img_path as input, processes it and returns a tuple (image, label).
  """
  image = img_to_tensor(img_path)
  return image, label

def create_batch(X, y=None, batch_size=BATCH_SIZE, test_data=False, valid_data=False):
  """
  Create a data batch of size same as batch_size.

  Shuffle data if it's training set and do not shuffle validation data. 
  Also accepts test data as input.
  """
  if test_data:
    print("Processing test data..")
    # Create a dataset from the data.
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))
    # Separate them into batches.
    data_batch = data.map(get_img_label).batch(batch_size)
  
  if valid_data:
    print("Processing validation data..")
    # Do not shuffle the data
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))
    data_batch = data.map(get_img_label).batch(batch_size)

  if not test_data and not valid_data:
    print("Processing training data..")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))
    # Shuffle the Data
    data = data.shuffle(buffer_size=len(X))
    data_batch = data.map(get_img_label).batch(batch_size)

  return data_batch

In [63]:
# Create training and validation data batches
train_data = create_batch(X_train, y_train)
val_data = create_batch(X_val, y_val, valid_data=True)

Processing training data..
Processing validation data..


In [66]:
test_data = create_batch(X_test, y_test, test_data=True)

Processing test data..


# Creating Models

In [14]:
# Setup Input Size
INPUT_SHAPE = (None, IMG_SIZE, IMG_SIZE, 3)

# Seutp output size
OUTPUT_SIZE = len(labels)

# Choosen model from tfHub
MODEL_URL = "https://tfhub.dev/sayakpaul/vit_s16_classification/1"

In [15]:
def create_model(input_shape=INPUT_SHAPE,output_shape=OUTPUT_SIZE,url=MODEL_URL):
  """
  Create, compile and build a model from tfHub.
  """
  print("Building model", url)
  
  model = tf.keras.Sequential([
    hub.KerasLayer(url),
    tf.keras.layers.Dense(units=output_shape,
                          activation="softmax")
  ])

  # Compile the model
  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      optimizer=tf.keras.optimizers.Adam(),
      metrics=["Accuracy"]
  )

  # Build model
  model.build(
      input_shape
  )

  return model

In [16]:
model = create_model()
model.summary()

Building model https://tfhub.dev/sayakpaul/vit_s16_classification/1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              22050664  
                                                                 
 dense (Dense)               (None, 43)                43043     
                                                                 
Total params: 22,093,707
Trainable params: 43,043
Non-trainable params: 22,050,664
_________________________________________________________________


# Train Model

In [17]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_Accuracy",
                                                  mode="max",
                                                  patience=1)

In [18]:
NUM_EPOCHS = 50 #@param {type:"slider",min:10,max:100,step:5}

In [19]:
def train_model():
  """
  Initialize a model and fit the data to the model
  """
  model = create_model()

  model.fit(x=train_data,
            validation_data=val_data,
            epochs=NUM_EPOCHS,
            validation_freq=1,
            callbacks=[early_stopping]
  )

  return model

In [20]:
# model = train_model()

Building model https://tfhub.dev/sayakpaul/vit_s16_classification/1
Epoch 1/50
66/66 [==============================] - 547s 8s/step - loss: 1.5251 - Accuracy: 0.6255 - val_loss: 0.4133 - val_Accuracy: 0.8617
Epoch 2/50
66/66 [==============================] - 502s 8s/step - loss: 0.2178 - Accuracy: 0.9479 - val_loss: 0.2004 - val_Accuracy: 0.9470
Epoch 3/50
66/66 [==============================] - 511s 8s/step - loss: 0.1096 - Accuracy: 0.9848 - val_loss: 0.1372 - val_Accuracy: 0.9773
Epoch 4/50
66/66 [==============================] - 502s 8s/step - loss: 0.0695 - Accuracy: 0.9905 - val_loss: 0.1158 - val_Accuracy: 0.9773


# Save Model

In [44]:
import os
import datetime

def save_model(model, suffix=None):
  """
  Saves a given model with along with suffix to a directory.
  """
  model_dir = os.path.join("/content/drive/MyDrive/Colab Notebooks/Recipeye/models",
                           datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  path = model_dir + suffix
  print(f"Saving model to: {path}")
  model.save(path)
  return path

In [45]:
# path = save_model(model, "ViT-Adam")

Saving model to: /content/drive/MyDrive/Colab Notebooks/Recipeye/models/20230131-145857ViT-Adam


# Load & test the model
Loading the saved model from the gdrive and testing the model on the test set.

In [69]:
loaded_model = tf.keras.models.load_model(path, custom_objects={"KerasLayer": hub.KerasLayer})

In [70]:
loaded_model

In [71]:
loaded_model.evaluate(test_data)

78/78 [==============================] - 496s 6s/step - loss: 0.4792 - Accuracy: 0.8483


[0.47923752665519714, 0.8482897281646729]